In [ ]:
from google.colab import drive

#drive.mount('/content/gdrive')


In [ ]:
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
torch.manual_seed(2)
np.random.seed(2)
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler, SequentialSampler
import torchvision.models as models
!pip install torchinfo
!pip install torchattacks
!pip install pip install grad-cam
!pip install ray
from torchinfo import summary
from torchvision.models.resnet import _resnet,BasicBlock
import torchattacks
import torchvision.utils
import torch.nn.functional as F
from pytorch_grad_cam import GradCAM
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

     |████████████████████████████████| 102 kB 4.1 MB/s 
     |████████████████████████████████| 4.5 MB 4.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for grad-cam: filename=grad_cam-1.3.7-py3-none-any.whl size=25953 sha256=cc23e34eb550112103e2f8056a9caa7c7ac03b6eef3d5a021ed174dbb5aab790
  Stored in directory: /root/.cache/pip/wheels/30/ab/9c/53c523785edffdc6c61755cf82e0dac3342d0d36190c187894
Successfully built grad-cam
     |████████████████████████████████| 59.6 MB 1.2 MB/s 
     |████████████████████████████████| 175 kB 43.5 MB/s 


In [ ]:
transform = transforms.Compose([
# transforms.Resize((224)),
transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(
root='./data', train=True, download=True, transform=transform)

valset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)
classes = ('plane', 'car', 'bird', 'cat', 'deer',
        'dog', 'frog', 'horse', 'ship', 'truck')


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
from torch.nn import Conv2d,AvgPool2d,Linear,Sequential,Dropout,BatchNorm2d,ModuleList,BatchNorm1d
import torch.nn.functional as F
import numpy as np
import math
from torch.autograd import Variable
from functools import partial

class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
            )
        self.pool_types = pool_types
    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type=='avg':
                avg_pool = F.avg_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( avg_pool )
            elif pool_type=='max':
                max_pool = F.max_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( max_pool )

            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = torch.sigmoid( channel_att_sum ).unsqueeze(2).unsqueeze(3).expand_as(x)
        return x * scale

def logsumexp_2d(tensor):
    tensor_flatten = tensor.view(tensor.size(0), tensor.size(1), -1)
    s, _ = torch.max(tensor_flatten, dim=2, keepdim=True)
    outputs = s + (tensor_flatten - s).exp().sum(dim=2, keepdim=True).log()
    return outputs

class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

class SpatialGate(nn.Module):
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = torch.sigmoid(x_out) # broadcasting
        return x * scale

class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
        super(CBAM, self).__init__()
        self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial=no_spatial
        if not no_spatial:
            self.SpatialGate = SpatialGate()
    def forward(self, x):
        x_out = self.ChannelGate(x)
        if not self.no_spatial:
            x_out = self.SpatialGate(x_out)
        return x_out

class Base(nn.Module):
    def freeze(self):
        for param in self.base_model.parameters():
                param.requires_grad = False
    
    def unfreeze(self):
        for param in self.base_model.parameters():
                param.requires_grad = True
    
    def attach_fea_out(self,classname,input,output):
        self.features.append(output)

    def attach_fea_in(self,classname,input,output):
        self.features.append(input[0])

    def __init__(self,trainable = True,attention=False,base=18):
        super(Base,self).__init__()
        self.features = []
        self.channel_size = []
        print(base)
        if base == 9:
            self.base_model = _resnet('resnet', BasicBlock, [1, 1, 1, 1], False, True)
        elif base==18:
            self.base_model = models.resnet18(pretrained=False)
        else:
            self.base_model = models.resnet34(pretrained=False)

        used_blocks = ['layer1', 'layer2','layer3','layer4']
        unused_blocks = ['avgpool','fc']

        for block in used_blocks:
            getattr(self.base_model,block).register_forward_hook(self.attach_fea_out)

        for block in unused_blocks:
             setattr(self.base_model,block,nn.Identity())
        
        if not trainable:
            self.freeze()

        fake_img = torch.rand(1,3,256,256) ## pass fake img to the model to get the channel size of each inception block
        self.base_model(fake_img)
        self.channel_size = [block.size()[1] for block in self.features]
        self.features = []

    def forward(self,img):
        self.base_model(img)

    def get_MLSP(self,img,feature_type,resize = True):
        self.base_model(img)
        if resize:
            print(resize)
            if feature_type == 'narrow':
                MLSP = [F.adaptive_avg_pool2d(block, (1, 1)) for block in self.features]
                for i in range(len(MLSP)):
                    MLSP[i] = MLSP[i].squeeze(2).squeeze(2)

            if feature_type == 'wide':
                MLSP = [F.interpolate(block,mode = 'bilinear', size = 7) for block in self.features]
            
            MLSP = torch.cat(MLSP,dim = 1)
            self.features = []
        else:
            MLSP = self.features
            self.features = []
        return MLSP



class Head(nn.Module):
    dp = 0.5
    def conv_block(self,inc,outc,ker,padding = 1,avgpool = False):
        modules = []
        modules.append(nn.Dropout(Head.dp))
        if avgpool:
            modules.append(AvgPool2d(3,1,1))
        modules.append(Conv2d(inc,outc,ker,padding = padding))
        modules.append(nn.BatchNorm2d(outc))
        modules.append(nn.ReLU())
        return Sequential(*modules)

    def __init__(self,head_type,num_channel):
        super(Head, self).__init__()
        self.head_type = head_type
        self.num_ch = num_channel

        if head_type == 'mlsp_cnn_gap_attn':
            self.attn = []
            self.conv = []
            for i in range(4):
                if i!=3:
                    self.attn.append(CBAM(num_channel[i],reduction_ratio=16))
                else:
                    self.attn.append(CBAM(num_channel[i],reduction_ratio=16,no_spatial=True))
                self.conv.append(Sequential(
                                    self.conv_block(num_channel[i],num_channel[i],1,0),
                                    self.conv_block(num_channel[i],num_channel[i],3,1),
                          ))
            self.attn = ModuleList(self.attn)
            self.conv = ModuleList(self.conv)
        self.dense = Sequential(Linear(960,10))

    def forward(self,features):
        if self.head_type == 'mlsp_gap':
            x = torch.cat([F.adaptive_avg_pool2d(feature, (1, 1)) for feature in features],dim=1)
        else:
            x = torch.cat([F.adaptive_avg_pool2d(block2(block1(feature)+feature),(1,1)) for feature,block1,block2 in zip(features,self.attn,self.conv)],dim=1)
        x = torch.flatten(x, 1)
        x = self.dense(x)
        return x

class Fmodel(nn.Module):
    def __init__(self, head_type='mlsp_gap',base = 18,dp = 0.5):
        super(Fmodel,self).__init__()
        self.bmodel = Base(base=base)
        Head.dp = dp
        self.head = Head(head_type,self.bmodel.channel_size)
        self.feature_type = 'narrow'    
        self.resize = False
        self.fea = []
        self.gap_fea = []
        self.gradient = []
        self.handles = []
        
    def forward(self,img):
        x = self.bmodel.get_MLSP(img,self.feature_type,self.resize)
        x = self.head(x)
        return x

    def unfreeze(self):
        self.bmodel.unfreeze()
    
    def freeze(self):
        self.bmodel.freeze()

    def hook_gap(self):
        handle = self.head.dense.register_forward_hook(lambda layer, inl, _,: self.gap_fea.append(inl[0]))
        self.handles += [handle]
        return handle

    def hook_grad(self):
        handle = []
        handle.append(self.head.conv[0].register_full_backward_hook(lambda layer, inl, out,: self.gradient.append(out[0])))
        handle.append(self.head.conv[1].register_full_backward_hook(lambda layer, inl, out,: self.gradient.append(out[0])))
        handle.append(self.head.conv[2].register_full_backward_hook(lambda layer, inl, out,: self.gradient.append(out[0])))
        handle.append(self.head.conv[3].register_full_backward_hook(lambda layer, inl, out,: self.gradient.append(out[0])))
        self.handles += handle
        return handle

    def hook_fea(self):
        handle = []
        handle.append(self.head.conv[0].register_forward_hook(lambda layer, inl, out,: self.fea.append(out)))
        handle.append(self.head.conv[1].register_forward_hook(lambda layer, inl, out,: self.fea.append(out)))
        handle.append(self.head.conv[2].register_forward_hook(lambda layer, inl, out,: self.fea.append(out)))
        handle.append(self.head.conv[3].register_forward_hook(lambda layer, inl, out,: self.fea.append(out)))
        self.handles += handle
        return handle

    def hook(self):
        self.hook_gap()
        self.hook_grad()
        self.hook_fea()

    def unhook(self):
        for fea in self.gap_fea:
            fea.detach()
        for grad in self.gradient:
            grad.detach()
        for fea in self.fea:
            fea.detach()

        self.gap_fea = []
        self.gradient = []
        self.fea = []

        for h in self.handles:
            h.remove()
        self.handles = []



In [ ]:
def grad_cam(model, preds, label,ind,ind_ad):
    masks_attn_ori = []
    masks_attn_adv = []


    preds[torch.arange(len(preds)),label].sum().backward(retain_graph=True)
    mse = torch.nn.MSELoss()
    loss = 0
    model.gradient = model.gradient[::-1]
    for i in range(4):
        weight = model.gradient[i].mean(dim=-1, keepdim=True).mean(dim=-2, keepdim=True)
        mask = weight * model.fea[i]
        mask_ori = F.relu(mask[ind].sum(dim=1))
        masks_attn_ori.append(mask_ori)
        
        mask_adv = F.relu(mask[ind_ad].sum(dim=1))      
        masks_attn_adv.append(mask_adv)

    model.zero_grad()
    for i in range(4):
        loss += mse(masks_attn_ori[i],masks_attn_adv[i])
    return loss

In [ ]:
from tqdm.auto import tqdm, trange
import torchattacks

def train_cifar(config, checkpoint_dir=None, data_dir=None):
    trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)

    valset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform)


    testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform)

    test_loader = torch.utils.data.DataLoader(
        testset, batch_size=64, shuffle=False, num_workers=2, sampler = SequentialSampler(testset.data[5000:10000]),pin_memory=True)
    
    num_epochs = 15 #50
    adv = True
    dense_l2 = True
    model = Fmodel('mlsp_cnn_gap_attn',18,config['dp'])
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['l2'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor=0.1, patience=1, verbose=True, min_lr =1e-6)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2,pin_memory=True)
    val_loader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False, num_workers=2, sampler = SequentialSampler(valset.data[5000:10000]),pin_memory=True)
    dataloaders = {"train": train_loader, "val": val_loader}

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    since = time.time()
    best_loss = 1000.0
    atk = torchattacks.PGD(model, eps=8/255, alpha=2/225, steps=7, random_start=True)
    atk_fgsm = torchattacks.FGSM(model, eps=8/255)
    
    print('Config: ', config)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']: # Each epoch has a training and validation phase
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            size = 0

            iterator = iter(dataloaders[phase])
            for _ in range(len(iterator)): # Iterate over data
                inputs, labels = next(iterator)
                if adv:
                    if attn_pre or ada_reg:
                            inputs = torch.cat((inputs.to(device), atk(inputs, labels)),dim=0)
                            labels = torch.cat((labels,labels),dim=0)
                            ind = list(range(len(inputs)//2))
                            ind_ad = list(range(len(inputs)//2,len(inputs)))
                    else:
                        ind_ad = np.random.choice(len(inputs),len(inputs)//2,replace=False)
                        ind = np.delete(np.arange(len(inputs)),ind_ad)
                        inputs_adv = atk(inputs[ind_ad], labels[ind_ad])
                        inputs = inputs.to(device)
                        inputs[ind_ad] = inputs_adv

                else:
                    inputs = inputs.to(device)

                inputs = transforms.functional.normalize(inputs,(0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                labels = labels.to(device)
                optimizer.zero_grad() # Zero the parameter gradients

                with torch.set_grad_enabled(phase == 'train'): # Forward. Track history if only in train
                    
                    if phase == 'train': # Backward + optimize only if in training phase
                        if dense_l2:
                            model.hook()
                        outputs = model(inputs)
                        loss = (config['adv_weight']*criterion(outputs[ind], labels[ind]) + (1-config['adv_weight'])*criterion(outputs[ind_ad], labels[ind_ad]))                           
                        _, preds = torch.max(outputs, 1)
                        loss_pres = grad_cam(model, outputs,labels,ind,ind_ad)
                        if not torch.isnan(loss_pres).any():
                            loss = config['scale']*loss + (1-config['scale'])*loss_pres
                        else:
                            print('nan')

                        if dense_l2:
                            fea = model.gap_fea[0]
                            out_ori = fea[ind]
                            out_adv = fea[ind_ad]
                            diff = (out_adv-out_ori).abs().mean(axis=0).reshape(out_ori.shape[-1])
                            diff = (diff - diff.min())/(diff.max()-diff.min()+1e-9)*config['ada_reg']
                            loss += (diff*((model.head.dense[0].weight)**2).sum(dim=0)).sum()

                        loss.backward()
                        optimizer.step()
                        if dense_l2:
                            model.unhook()
                    
                    if phase == 'val':
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)

                
                # Statistics
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                size+=len(preds)

            epoch_loss = running_loss / size
            
            
            if phase == 'train': # Adjust learning rate based on val loss
                scheduler.step(epoch_loss)
                
            epoch_acc = running_corrects.double() / size
            #print('{} Loss: {:.3f} Loss2: {:.3f} Acc: {:.3f}'.format(phase, epoch_loss, np.mean(loss2_his), epoch_acc))
            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                print('Best loss: ', best_loss)
            if phase == 'val':
                with tune.checkpoint_dir(epoch) as checkpoint_dir:
                    path = os.path.join(checkpoint_dir, "checkpoint")
                    torch.save((model.state_dict(), optimizer.state_dict()), path)

                tune.report(loss=epoch_loss, accuracy=epoch_acc)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best_loss: {:4f}'.format(best_loss))



In [ ]:
from functools import partial

config = {
    "dp": tune.choice([0.5,0.4,0.3, 0.2,0.1]),
    "l2": tune.loguniform(1e-5, 1e-2),
    "ada_reg": tune.loguniform(1e-5, 1e-2),
    "adv_weight": tune.uniform(0.4, 0.6),
    "scale": tune.uniform(0.4, 0.6),
    "lr": tune.loguniform(1e-4, 1e-2),
}
reporter = CLIReporter(
    # parameter_columns=["l1", "l2", "lr", "batch_size"],
    metric_columns=["loss", "accuracy",'training iteration'])

scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=20,
        grace_period=1,
        reduction_factor=2)

result = tune.run(
    partial(train_cifar),
    resources_per_trial={"cpu": 2, "gpu": 1},
    config=config,
    num_samples=30,
    verbose=0,
    scheduler=scheduler,
    progress_reporter=reporter,
)

best_trial = result.get_best_trial("loss", "min", 'all')
print("Best trial config: {}".format(best_trial.config))



2022-03-08 11:13:53,646	WARNING experiment.py:256 -- No name detected on trainable. Using DEFAULT.
2022-03-08 11:13:53,656	INFO registry.py:70 -- Detected unknown callable for trainable. Converting to class.
2022-03-08 11:13:53,807	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(func pid=298) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz


 97%|█████████▋| 165266432/170498071 [00:10<00:00, 17110407.17it/s]
170499072it [00:10, 15548277.13it/s]                               


(func pid=298) Extracting ./data/cifar-10-python.tar.gz to ./data
(func pid=298) Files already downloaded and verified
(func pid=298) Files already downloaded and verified
(func pid=298) 18


(func pid=298) E0308 11:14:22.415531481     380 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=298) E0308 11:14:22.438941128     380 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


...

(func pid=757) E0308 13:52:47.905639049     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=757) E0308 13:52:47.931556566     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=757) E0308 14:00:50.648202139     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=757) E0308 14:00:50.673751633     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


(func pid=757) Best loss:  1.0370936120986938
(func pid=757) Epoch 4/14
(func pid=757) ----------


(func pid=757) E0308 14:01:06.174220559     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=757) E0308 14:01:06.203371350     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=757) E0308 14:09:07.014249444     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=757) E0308 14:09:07.039616833     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


(func pid=757) Best loss:  1.0200728635787963
(func pid=757) Epoch 5/14
(func pid=757) ----------


(func pid=757) E0308 14:09:22.572624602     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=757) E0308 14:09:22.606160108     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=757) E0308 14:17:23.044836949     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(func pid=757) E0308 14:17:23.070174068     785 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [ ]:
best_trial = result.get_best_trial("loss", "min", 'all')
print("Best trial config: {}".format(best_trial.config))

Best trial config: {'dp': 0.1, 'l2': 5.467130776888194e-06, 'ada_reg': 0.008446479631052932, 'adv_weight': 0.73428888168777, 'scale': 0.5988473674763856, 'lr': 0.004933134661531971}
Best trial final validation loss: 0.9047795356750489
Best trial final validation accuracy: 0.7273000000000001
